# Aprendizaje Automático 1
# IEFI

ALUMNO: Nahuel Lahoz 

## Primera Parte - Imputación

- Se le da el dataset real correspondiente a un Banco de Japón, *japan_bank_500_conNulos.csv*. En este dataset la variable target es la y, en cuanto a las features las hay numéricas y categóricas.
- El banco no nos proveyó de una descripción de qué es cada variable (habitual en estos casos!) .  

- El dataset tiene muchos nulos.   

- Su primer tarea es imputar algún tipo de valor a los nulos de cada feature. 

Para ello:  

- Divida  hasta Validation Train y Validation Test. Cada división deberá hacerse 80/20 y con semilla 123.

- A nivel de Validation probará **distintas formas de imputar los nulos** (por ejemplo imputar media o mediana etc) y las evaluará en un **único modelo**  que será árbol con 4 niveles de profundidad en todos los casos. Tenga en cuenta que las variables categóricas deberán pasar por OneHotEncoder para ser útiles y la misma target posiblemente también. 

- La estrategia de imputación que le de mejor resultado de Accuracy con el mencionado árbol entrenando en el Validation Train y evaluando en el Validation Test será la forma de imputación elegida para la segunda parte ya que asumiremos que es la mejor para este dataset. 

En la celda siguiente escriba cuál fue la estrategia de imputación elegida y los valores de Accuracy obtenidos en cada caso. Si le resulta incómodo hacerlo aquí, puede escribir el resumen en Word. 


El imputador elegido es el 3. Este tiene la siguientes carcateristicas:
* A los missing values numericos les imputa el valor a traves del Knn imputer con vecinos 5.
* A los valores categoricos, les asigna la moda
* Accuracy: 0.7875


Respuesta:


In [1]:
#Importaciones
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer #Imputador
from sklearn.model_selection import train_test_split #Split df
from sklearn.compose import ColumnTransformer #Transformador de columnas
from sklearn.preprocessing import OneHotEncoder #Dummy categoricas
from sklearn.impute import KNNImputer #Imputador kNn
from sklearn.tree import DecisionTreeClassifier #arbol de decision
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score #Metricas
from sklearn.preprocessing import MinMaxScaler #Normalizador
from sklearn import neighbors #KNN
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression #Regresion logistoca

# sólo para que no muestre el Warning
import warnings
warnings.simplefilter('ignore')

In [2]:
df=pd.read_csv("data\japan_bank_500_conNulos.csv") #cargo el df


In [ ]:
#df.describe(include='all')

In [ ]:
#df.info()

In [4]:
#Analisis de cada columna
df_obj=df.select_dtypes(object,bool)
for (columnName, columnData) in df_obj.iteritems():
        print('Colunm Name : ', columnName)
        print('Column Contents : ',columnData.unique())



Colunm Name :  x1
Column Contents :  ['b' nan 'a']
Colunm Name :  x4
Column Contents :  [nan 'y' 'u' 'l']
Colunm Name :  x5
Column Contents :  ['g' 'p' nan 'gg']
Colunm Name :  x6
Column Contents :  [nan 'ff' 'c' 'cc' 'm' 'w' 'i' 'aa' 'j' 'e' 'x' 'q' 'd' 'k' 'r']
Colunm Name :  x7
Column Contents :  ['v' nan 'ff' 'h' 'bb' 'j' 'n' 'z' 'dd' 'o']
Colunm Name :  x9
Column Contents :  ['f' 't' nan]
Colunm Name :  x10
Column Contents :  ['f' nan 't']
Colunm Name :  x12
Column Contents :  ['f' 't' nan]
Colunm Name :  x13
Column Contents :  [nan 'g' 's' 'p']
Colunm Name :  y
Column Contents :  ['-' '+']


In [5]:
#Luego de obtener los unique de cada columna categorica, creo estas variables donde guardo su valor
#para pasarselo despues al O.H.E

v_x1=['b','a']
v_x4=[ 'y', 'u', 'l']
v_x5=['g', 'p' ,'gg']
v_x6=['ff', 'c', 'cc', 'm', 'w' ,'i' ,'aa' ,'j', 'e', 'x', 'q', 'd', 'k', 'r']
v_x7=['v' , 'ff' ,'h', 'bb', 'j', 'n' ,'z' ,'dd' ,'o']
v_x9=['f' ,'t' ]
v_x10=['f' ,'t' ]
v_x12=['f' ,'t' ]
v_x13=[ 'g', 's' ,'p']


In [6]:

df_num=df.select_dtypes(np.number)
for (columnName, columnData) in df_num.iteritems():
        print('Colunm Name : ', columnName)
        print('Column Contents : ',columnData.describe())



Colunm Name :  x2
Column Contents :  count    295.000000
mean      31.311729
std       11.501261
min       13.750000
25%       22.625000
50%       28.580000
75%       36.960000
max       80.250000
Name: x2, dtype: float64
Colunm Name :  x3
Column Contents :  count      317.000000
mean      1181.044826
std       3456.727099
min          0.000000
25%          1.710000
50%          6.500000
75%        165.000000
max      25125.000000
Name: x3, dtype: float64
Colunm Name :  x8
Column Contents :  count      311.000000
mean       384.783794
std       1349.843628
min          0.000000
25%          0.500000
50%          2.540000
75%        125.000000
max      13875.000000
Name: x8, dtype: float64
Colunm Name :  x11
Column Contents :  count    293.000000
mean       2.252560
std        3.975658
min        0.000000
25%        0.000000
50%        0.000000
75%        3.000000
max       19.000000
Name: x11, dtype: float64
Colunm Name :  x14
Column Contents :  count     289.000000
mean      199.53287

In [7]:
#Preparacion de los datos para split
x=df.drop(axis=1, labels='y')   #guardamos las variables para X
y=df[['y']]                     #guardamos las variables para y


In [8]:
#split en test train 
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=123)

#Validations
X_val_train,X_val_test,y_val_train,y_val_test=train_test_split(X_train,y_train,test_size=0.20,random_state=123)


In [9]:
##CONVERTIMOS Y EN VALORES NUMERICOS

y_train=y_train['y'].map({'-':0,'+':1})
y_train= y_train.to_numpy()

y_test=y_test['y'].map({'-':0,'+':1})
y_test= y_test.to_numpy()

y_val_train=y_val_train['y'].map({'-':0,'+':1})
y_val_train= y_val_train.to_numpy()

y_val_test=y_val_test['y'].map({'-':0,'+':1})
y_val_test= y_val_test.to_numpy() 

In [12]:
#Transofrmadores numericos
cols_num=X_val_train.select_dtypes(np.number).columns 
t1=('imputador_num', SimpleImputer(strategy='mean',copy=True), cols_num)
t2=('imputador_num', SimpleImputer(strategy='median',copy=True), cols_num)
t3=('imputador_num', KNNImputer(n_neighbors=5,copy=True), cols_num)
t4=('imputador_num', KNNImputer(n_neighbors=5, weights='distance',copy=True), cols_num)



#Transfromadores categoricos
cols_cat=X_val_train.select_dtypes(include=['object','bool']).columns

t5=('imputador_cat', SimpleImputer(strategy='most_frequent',copy=True), cols_cat) 



In [13]:
#Transformador de columnas
transformador_columnas1 = ColumnTransformer(transformers=[t1,t5],remainder='passthrough')  
transformador_columnas2 = ColumnTransformer(transformers=[t2,t5],remainder='passthrough') 
transformador_columnas3 = ColumnTransformer(transformers=[t3,t5],remainder='passthrough') 
transformador_columnas4 = ColumnTransformer(transformers=[t4,t5],remainder='passthrough') 

### Transformador 1

In [15]:
# Lo entrenamos con el Validation Train:
transformador_columnas1.fit(X_val_train)

X_val_Train_Array = transformador_columnas1.transform(X_val_train)
X_val_Test_Array = transformador_columnas1.transform(X_val_test)


In [16]:
# Ordenamos los data frame por las columnas

cols_orden=cols_num.tolist() + cols_cat.tolist()

X_val_train_Transf=pd.DataFrame(X_val_Train_Array, columns=cols_orden) 
X_val_test_Transf=pd.DataFrame(X_val_Test_Array, columns=cols_orden)


In [17]:
# Definimos el transformador para OneHotEncoder 
t_dummy=("onehot",OneHotEncoder(categories=[v_x1,v_x4,v_x5,v_x6,v_x7,v_x9,v_x10,v_x12,v_x13] ,
                                sparse=False), ['x1','x4','x5','x6','x7','x9','x10','x12','x13'])

#Normalizador
t_normalizador=("normalizador",MinMaxScaler(feature_range=(0, 1),copy=True),cols_num)

# Creamos el transformador de columnas
transformador_norm_dummy = ColumnTransformer(transformers=[t_dummy,t_normalizador],remainder='passthrough')
# Lo entrenamos con fit en df
transformador_norm_dummy.fit(X_val_train_Transf)
# Lo aplicamos al df
dummy_X_val_train=transformador_norm_dummy.transform(X_val_train_Transf)


# Lo aplicamos a X_val_test_Transformado
dummy_X_val_test=transformador_norm_dummy.transform(X_val_test_Transf)

### Arbol imputador1

In [18]:
# Eleccion de imputacion a valores nulos

Arbol_P=DecisionTreeClassifier(max_depth=4, random_state=123)

# Entrenamos el modelo en el validation
Arbol_P.fit(dummy_X_val_train, y_val_train) 


# Pronosticamos los valores de y para los valores del X Val test
y_val_pred = Arbol_P.predict(dummy_X_val_test)


# Evaluamos en el Val Test
ACTree=accuracy_score(y_val_test,y_val_pred)


# Mostramos resultados
print("Evaluación modelo Árbol de Decisión en el Imputador 1")
print('****************************************************')     
print('Accuracy:', round(ACTree,4))

Evaluación modelo Árbol de Decisión en el Imputador 1
****************************************************
Accuracy: 0.7125


### Transformador 2

In [19]:
# Lo entrenamos con el Validation Train:
transformador_columnas2.fit(X_val_train)

X_val_Train_Array = transformador_columnas2.transform(X_val_train)
X_val_Test_Array = transformador_columnas2.transform(X_val_test)

In [20]:
# Ordenamos los data frame por las columnas

cols_orden=cols_num.tolist() + cols_cat.tolist()

X_val_train_Transf=pd.DataFrame(X_val_Train_Array, columns=cols_orden) 
X_val_test_Transf=pd.DataFrame(X_val_Test_Array, columns=cols_orden)


In [21]:
# Definimos el transformador para OneHotEncoder 
t_dummy=("onehot",OneHotEncoder(categories=[v_x1,v_x4,v_x5,v_x6,v_x7,v_x9,v_x10,v_x12,v_x13] ,
                                sparse=False), ['x1','x4','x5','x6','x7','x9','x10','x12','x13'])

#Normalizador
t_normalizador=("normalizador",MinMaxScaler(feature_range=(0, 1),copy=True),cols_num)

# Creamos el transformador de columnas
transformador_norm_dummy = ColumnTransformer(transformers=[t_dummy,t_normalizador],remainder='passthrough')
# Lo entrenamos con fit en df
transformador_norm_dummy.fit(X_val_train_Transf)
# Lo aplicamos al df
dummy_X_val_train=transformador_norm_dummy.transform(X_val_train_Transf)


# Lo aplicamos a X_val_test_Transformado
dummy_X_val_test=transformador_norm_dummy.transform(X_val_test_Transf)


### Arbol Imputador2

In [22]:
# Eleccion de imputacion a valores nulos

Arbol_P=DecisionTreeClassifier(max_depth=4, random_state=123)

# Entrenamos el modelo en el validation
Arbol_P.fit(dummy_X_val_train, y_val_train) 


# Pronosticamos los valores de y para los valores del X Val test
y_val_pred = Arbol_P.predict(dummy_X_val_test)


# Evaluamos en el Val Test
ACTree=accuracy_score(y_val_test,y_val_pred)


# Mostramos resultados
print("Evaluación modelo Árbol de Decisión en el Imputador 2")
print('****************************************************')     
print('Accuracy:', round(ACTree,4))

Evaluación modelo Árbol de Decisión en el Imputador 2
****************************************************
Accuracy: 0.7375


### Tranformador 3

In [23]:
# Lo entrenamos con el Validation Train:
transformador_columnas3.fit(X_val_train)

X_val_Train_Array = transformador_columnas3.transform(X_val_train)
X_val_Test_Array = transformador_columnas3.transform(X_val_test)

In [24]:
# Ordenamos los data frame por las columnas

cols_orden=cols_num.tolist() + cols_cat.tolist()

X_val_train_Transf=pd.DataFrame(X_val_Train_Array, columns=cols_orden) 
X_val_test_Transf=pd.DataFrame(X_val_Test_Array, columns=cols_orden)


In [25]:
# Definimos el transformador para OneHotEncoder 
t_dummy=("onehot",OneHotEncoder(categories=[v_x1,v_x4,v_x5,v_x6,v_x7,v_x9,v_x10,v_x12,v_x13] ,
                                sparse=False), ['x1','x4','x5','x6','x7','x9','x10','x12','x13'])

#Normalizador
t_normalizador=("normalizador",MinMaxScaler(feature_range=(0, 1),copy=True),cols_num)

# Creamos el transformador de columnas
transformador_norm_dummy = ColumnTransformer(transformers=[t_dummy,t_normalizador],remainder='passthrough')
# Lo entrenamos con fit en df
transformador_norm_dummy.fit(X_val_train_Transf)
# Lo aplicamos al df
dummy_X_val_train=transformador_norm_dummy.transform(X_val_train_Transf)


# Lo aplicamos a X_val_test_Transformado
dummy_X_val_test=transformador_norm_dummy.transform(X_val_test_Transf)

### Arbol imputador3

In [22]:
# Eleccion de imputacion a valores nulos

Arbol_P=DecisionTreeClassifier(max_depth=4, random_state=123)

# Entrenamos el modelo en el validation
Arbol_P.fit(dummy_X_val_train, y_val_train) 


# Pronosticamos los valores de y para los valores del X Val test
y_val_pred = Arbol_P.predict(dummy_X_val_test)


# Evaluamos en el Val Test
ACTree=accuracy_score(y_val_test,y_val_pred)


# Mostramos resultados
print("Evaluación modelo Árbol de Decisión en el Imputador 3")
print('****************************************************')     
print('Accuracy:', round(ACTree,4))

Evaluación modelo Árbol de Decisión en el Imputador 3
****************************************************
Accuracy: 0.7875


### Transformador 4

In [26]:
# Lo entrenamos con el Validation Train:
transformador_columnas4.fit(X_val_train)

X_val_Train_Array = transformador_columnas4.transform(X_val_train)
X_val_Test_Array = transformador_columnas4.transform(X_val_test)

In [27]:
# Ordenamos los data frame por las columnas

cols_orden=cols_num.tolist() + cols_cat.tolist()

X_val_train_Transf=pd.DataFrame(X_val_Train_Array, columns=cols_orden) 
X_val_test_Transf=pd.DataFrame(X_val_Test_Array, columns=cols_orden)


In [28]:
# Definimos el transformador para OneHotEncoder 
t_dummy=("onehot",OneHotEncoder(categories=[v_x1,v_x4,v_x5,v_x6,v_x7,v_x9,v_x10,v_x12,v_x13] ,
                                sparse=False), ['x1','x4','x5','x6','x7','x9','x10','x12','x13'])

#Normalizador
t_normalizador=("normalizador",MinMaxScaler(feature_range=(0, 1),copy=True),cols_num)

# Creamos el transformador de columnas
transformador_norm_dummy = ColumnTransformer(transformers=[t_dummy,t_normalizador],remainder='passthrough')
# Lo entrenamos con fit en df
transformador_norm_dummy.fit(X_val_train_Transf)
# Lo aplicamos al df
dummy_X_val_train=transformador_norm_dummy.transform(X_val_train_Transf)


# Lo aplicamos a X_val_test_Transformado
dummy_X_val_test=transformador_norm_dummy.transform(X_val_test_Transf)

### Arbol Imputador4

In [26]:
# Eleccion de imputacion a valores nulos

Arbol_P=DecisionTreeClassifier(max_depth=4, random_state=123)

# Entrenamos el modelo en el validation
Arbol_P.fit(dummy_X_val_train, y_val_train) 


# Pronosticamos los valores de y para los valores del X Val test
y_val_pred = Arbol_P.predict(dummy_X_val_test)


# Evaluamos en el Val Test
ACTree=accuracy_score(y_val_test,y_val_pred)


# Mostramos resultados
print("Evaluación modelo Árbol de Decisión en el Imputador 4")
print('****************************************************')     
print('Accuracy:', round(ACTree,4))

Evaluación modelo Árbol de Decisión en el Imputador 4
****************************************************
Accuracy: 0.7125


## Segunda Parte:  

Una vez decidido cómo imputará deberá **elegir el modelo e hiperparámetros** que mejor sirve para pronosticar los valores del target.   
- Tenga en cuenta que como hará selección de modelo e hiperparámetros deberá compararlos en el Validation.    

- Una vez elegido el mejor modelo, deberá indicar cuál es la Accuracy Esperada cuando el modelo se use con nuevas observaciones. 

 En la celda siguiente indique cuál fue:  
 
 - modelo elegido:  Arbol de Decisión
 - hiperparámetros: Profundidad= 4 
 - Accuracy Esperada: 0.7875

### Imputacion de los valores con parametros seleccionados

A esta altura el df ya esta cargado, dividido en x e y y en train-test.
Sin embargo las imputaciones y transformaciones no se aplicaron al df por  el copy=True.



In [29]:
#Transofrmadores numericos
cols_num=X_val_train.select_dtypes(np.number).columns 
t3=('imputador_num', KNNImputer(n_neighbors=5), cols_num)

#Transfromadores categoricos
cols_cat=X_val_train.select_dtypes(include=['object','bool']).columns

t5=('imputador_cat', SimpleImputer(strategy='most_frequent'), cols_cat) 

In [30]:
transformador_columnas3 = ColumnTransformer(transformers=[t3,t5],remainder='passthrough') 

In [31]:
# Lo entrenamos con el Validation Train:
transformador_columnas3.fit(X_val_train)

X_val_Train_Array = transformador_columnas3.transform(X_val_train)
X_val_Test_Array = transformador_columnas3.transform(X_val_test)

In [32]:
# Ordenamos los data frame por las columnas

cols_orden=cols_num.tolist() + cols_cat.tolist()

X_val_train_Transf=pd.DataFrame(X_val_Train_Array, columns=cols_orden) 
X_val_test_Transf=pd.DataFrame(X_val_Test_Array, columns=cols_orden)


In [33]:
# Definimos el transformador para OneHotEncoder 
t_dummy=("onehot",OneHotEncoder(categories=[v_x1,v_x4,v_x5,v_x6,v_x7,v_x9,v_x10,v_x12,v_x13] ,
                                sparse=False), ['x1','x4','x5','x6','x7','x9','x10','x12','x13'])

#Normalizador
t_normalizador=("normalizador",MinMaxScaler(feature_range=(0, 1)),cols_num)

# Creamos el transformador de columnas
transformador_norm_dummy = ColumnTransformer(transformers=[t_dummy,t_normalizador],remainder='passthrough')
# Lo entrenamos con fit en df
transformador_norm_dummy.fit(X_val_train_Transf)
# Lo aplicamos al df
dummy_X_val_train=transformador_norm_dummy.transform(X_val_train_Transf)


# Lo aplicamos a X_val_test_Transformado
dummy_X_val_test=transformador_norm_dummy.transform(X_val_test_Transf)

## Prueba de modelos

### Arbol con diferentes profundidades

In [34]:
for profundidad in range(1,15):
    arbol=DecisionTreeClassifier(max_depth=profundidad, random_state=123)
    arbol.fit(dummy_X_val_train,y_val_train)
    y_val_pred=arbol.predict(dummy_X_val_test)
    ACTree=accuracy_score(y_val_test,y_val_pred)
    print("Accuracy de Arbol de decision de profundidad", profundidad,":", ACTree)
    print('****************************************************')

Accuracy de Arbol de decision de profundidad 1 : 0.7625
****************************************************
Accuracy de Arbol de decision de profundidad 2 : 0.7875
****************************************************
Accuracy de Arbol de decision de profundidad 3 : 0.775
****************************************************
Accuracy de Arbol de decision de profundidad 4 : 0.7875
****************************************************
Accuracy de Arbol de decision de profundidad 5 : 0.7125
****************************************************
Accuracy de Arbol de decision de profundidad 6 : 0.675
****************************************************
Accuracy de Arbol de decision de profundidad 7 : 0.7125
****************************************************
Accuracy de Arbol de decision de profundidad 8 : 0.6625
****************************************************
Accuracy de Arbol de decision de profundidad 9 : 0.6625
****************************************************
Accuracy de Arbol de 

### kNn con uniform y distancia Manhattan

In [35]:
#Modelo con peso uniforme

for n in range (1,15):
    Ku_mod=neighbors.KNeighborsClassifier(n_neighbors=n, weights='uniform',metric='minkowski',p=1)
    Ku_mod.fit(dummy_X_val_train,y_val_train)
    yval_predK=Ku_mod.predict(dummy_X_val_test)    
    ACkNn=accuracy_score(y_val_test,yval_predK)
    print("Accuracy de KnN con vecinos", n ,":", ACkNn)
    print('****************************************************')

Accuracy de KnN con vecinos 1 : 0.7
****************************************************
Accuracy de KnN con vecinos 2 : 0.7
****************************************************
Accuracy de KnN con vecinos 3 : 0.7
****************************************************
Accuracy de KnN con vecinos 4 : 0.675
****************************************************
Accuracy de KnN con vecinos 5 : 0.7125
****************************************************
Accuracy de KnN con vecinos 6 : 0.675
****************************************************
Accuracy de KnN con vecinos 7 : 0.7125
****************************************************
Accuracy de KnN con vecinos 8 : 0.675
****************************************************
Accuracy de KnN con vecinos 9 : 0.7
****************************************************
Accuracy de KnN con vecinos 10 : 0.6375
****************************************************
Accuracy de KnN con vecinos 11 : 0.6375
****************************************************
A

### kNn con uniform y distancia Euclidean

In [34]:
for n in range (1,15):
    Ku_mod=neighbors.KNeighborsClassifier(n_neighbors=n, weights='uniform',metric='minkowski',p=2)
    Ku_mod.fit(dummy_X_val_train,y_val_train)
    yval_predK=Ku_mod.predict(dummy_X_val_test)    
    ACkNn=accuracy_score(y_val_test,yval_predK)
    print("Accuracy de KnN con vecinos", n ,":", ACkNn)
    print('****************************************************')

Accuracy de KnN con vecinos 1 : 0.7
****************************************************
Accuracy de KnN con vecinos 2 : 0.7125
****************************************************
Accuracy de KnN con vecinos 3 : 0.725
****************************************************
Accuracy de KnN con vecinos 4 : 0.7125
****************************************************
Accuracy de KnN con vecinos 5 : 0.725
****************************************************
Accuracy de KnN con vecinos 6 : 0.7125
****************************************************
Accuracy de KnN con vecinos 7 : 0.7
****************************************************
Accuracy de KnN con vecinos 8 : 0.6875
****************************************************
Accuracy de KnN con vecinos 9 : 0.7125
****************************************************
Accuracy de KnN con vecinos 10 : 0.6625
****************************************************
Accuracy de KnN con vecinos 11 : 0.6875
***********************************************

### kNn con uniform y distancia Minkowski =4

In [35]:
for n in range (1,15):
    Ku_mod=neighbors.KNeighborsClassifier(n_neighbors=n, weights='uniform',metric='minkowski',p=4)
    Ku_mod.fit(dummy_X_val_train,y_val_train)
    yval_predK=Ku_mod.predict(dummy_X_val_test)    
    ACkNn=accuracy_score(y_val_test,yval_predK)
    print("Accuracy de KnN con vecinos", n ,":", ACkNn)
    print('****************************************************')

Accuracy de KnN con vecinos 1 : 0.6875
****************************************************
Accuracy de KnN con vecinos 2 : 0.725
****************************************************
Accuracy de KnN con vecinos 3 : 0.7375
****************************************************
Accuracy de KnN con vecinos 4 : 0.725
****************************************************
Accuracy de KnN con vecinos 5 : 0.75
****************************************************
Accuracy de KnN con vecinos 6 : 0.7125
****************************************************
Accuracy de KnN con vecinos 7 : 0.7
****************************************************
Accuracy de KnN con vecinos 8 : 0.6875
****************************************************
Accuracy de KnN con vecinos 9 : 0.7
****************************************************
Accuracy de KnN con vecinos 10 : 0.7
****************************************************
Accuracy de KnN con vecinos 11 : 0.7
****************************************************
Ac

### kNn distance, con manhattan

In [36]:


for n in range (1,15):
    Kd_mod=neighbors.KNeighborsClassifier(n_neighbors=n,weights='distance',metric='minkowski',p=1 )
    Kd_mod.fit(dummy_X_val_train,y_val_train)
    y_val_predD=Kd_mod.predict(dummy_X_val_test)
    ACkd=accuracy_score(y_val_test,y_val_predD)
    print("Accuracy de KnN con vecinos", n ,":", ACkd)
    print('****************************************************')


Accuracy de KnN con vecinos 1 : 0.7
****************************************************
Accuracy de KnN con vecinos 2 : 0.7
****************************************************
Accuracy de KnN con vecinos 3 : 0.6875
****************************************************
Accuracy de KnN con vecinos 4 : 0.7125
****************************************************
Accuracy de KnN con vecinos 5 : 0.7125
****************************************************
Accuracy de KnN con vecinos 6 : 0.6875
****************************************************
Accuracy de KnN con vecinos 7 : 0.7125
****************************************************
Accuracy de KnN con vecinos 8 : 0.7125
****************************************************
Accuracy de KnN con vecinos 9 : 0.675
****************************************************
Accuracy de KnN con vecinos 10 : 0.6625
****************************************************
Accuracy de KnN con vecinos 11 : 0.6625
**********************************************

### kNn distance, con euclidean

In [37]:

for n in range (1,15):
    Kd_mod=neighbors.KNeighborsClassifier(n_neighbors=n,weights='distance',metric='minkowski',p=2 )
    Kd_mod.fit(dummy_X_val_train,y_val_train)
    y_val_predD=Kd_mod.predict(dummy_X_val_test)
    ACkd=accuracy_score(y_val_test,y_val_predD)
    print("Accuracy de KnN con vecinos", n ,":", ACkd)
    print('****************************************************')

Accuracy de KnN con vecinos 1 : 0.7
****************************************************
Accuracy de KnN con vecinos 2 : 0.7
****************************************************
Accuracy de KnN con vecinos 3 : 0.7
****************************************************
Accuracy de KnN con vecinos 4 : 0.7
****************************************************
Accuracy de KnN con vecinos 5 : 0.725
****************************************************
Accuracy de KnN con vecinos 6 : 0.7
****************************************************
Accuracy de KnN con vecinos 7 : 0.7
****************************************************
Accuracy de KnN con vecinos 8 : 0.7125
****************************************************
Accuracy de KnN con vecinos 9 : 0.7
****************************************************
Accuracy de KnN con vecinos 10 : 0.7
****************************************************
Accuracy de KnN con vecinos 11 : 0.675
****************************************************
Accuracy de 

### kNn distance, con Minkowski=4

In [38]:

for n in range (1,15):
    Kd_mod=neighbors.KNeighborsClassifier(n_neighbors=n,weights='distance',metric='minkowski',p=4 )
    Kd_mod.fit(dummy_X_val_train,y_val_train)
    y_val_predD=Kd_mod.predict(dummy_X_val_test)
    ACkd=accuracy_score(y_val_test,y_val_predD)
    print("Accuracy de KnN con vecinos", n ,":", ACkd)
    print('****************************************************')

Accuracy de KnN con vecinos 1 : 0.6875
****************************************************
Accuracy de KnN con vecinos 2 : 0.6875
****************************************************
Accuracy de KnN con vecinos 3 : 0.7125
****************************************************
Accuracy de KnN con vecinos 4 : 0.7375
****************************************************
Accuracy de KnN con vecinos 5 : 0.75
****************************************************
Accuracy de KnN con vecinos 6 : 0.725
****************************************************
Accuracy de KnN con vecinos 7 : 0.6875
****************************************************
Accuracy de KnN con vecinos 8 : 0.725
****************************************************
Accuracy de KnN con vecinos 9 : 0.6875
****************************************************
Accuracy de KnN con vecinos 10 : 0.7125
****************************************************
Accuracy de KnN con vecinos 11 : 0.675
********************************************

### Regresion logistica

In [36]:
modelo=LogisticRegression(penalty='none', random_state=123,multi_class='ovr')
modelo.fit(dummy_X_val_train,y_val_train)
probas=modelo.predict_proba(dummy_X_val_test)

In [37]:
umbral_0=0.54 # que pronostique 0 si la probabilidad es mayor que 0.54
y_pred_umbral=[]

for i in np.arange(0,probas.shape[0]):
    if probas[i][0]>umbral_0:
        y_pred_agregar=0
    else:
        y_pred_agregar=1
    
    y_pred_umbral.append(y_pred_agregar)
    
ACC_umbral=accuracy_score(y_val_test,y_pred_umbral)
print("El accuracy de Regresion Logistica con umbral:",umbral_0, "es de:" ,ACC_umbral)


El accuracy de Regresion Logistica con umbral: 0.54 es de: 0.75


## Evaluación del modelo

### Preparacion de los datos

In [38]:
#Entreno en el train, testeo en el test
#Primero preparo los datos


#Transofrmadores numericos
cols_num=X_train.select_dtypes(np.number).columns 
t3=('imputador_num', KNNImputer(n_neighbors=5), cols_num)

#Transfromadores categoricos
cols_cat=X_train.select_dtypes(include=['object','bool']).columns

t5=('imputador_cat', SimpleImputer(strategy='most_frequent'), cols_cat) 

In [39]:
transformador_columnas3 = ColumnTransformer(transformers=[t3,t5],remainder='passthrough') 

In [40]:
# Lo entrenamos con el Validation Train:
transformador_columnas3.fit(X_train)

X_Train_Array = transformador_columnas3.transform(X_train)
X_Test_Array = transformador_columnas3.transform(X_test)

In [41]:
# Ordenamos los data frame por las columnas

cols_orden=cols_num.tolist() + cols_cat.tolist()

X_train_Transf=pd.DataFrame(X_Train_Array, columns=cols_orden) 
X_test_Transf=pd.DataFrame(X_Test_Array, columns=cols_orden)


In [42]:
# Definimos el transformador para OneHotEncoder 
t_dummy=("onehot",OneHotEncoder(categories=[v_x1,v_x4,v_x5,v_x6,v_x7,v_x9,v_x10,v_x12,v_x13] ,
                                sparse=False), ['x1','x4','x5','x6','x7','x9','x10','x12','x13'])

#Normalizador
t_normalizador=("normalizador",MinMaxScaler(feature_range=(0, 1)),cols_num)

# Creamos el transformador de columnas
transformador_norm_dummy = ColumnTransformer(transformers=[t_dummy,t_normalizador],remainder='passthrough')
# Lo entrenamos con fit en df
transformador_norm_dummy.fit(X_train_Transf)
# Lo aplicamos al df
dummy_X_train=transformador_norm_dummy.transform(X_train_Transf)


# Lo aplicamos a X_test_Transformado
dummy_X_test=transformador_norm_dummy.transform(X_test_Transf)

### Modelo elegido

In [47]:
arbol=DecisionTreeClassifier(max_depth=4, random_state=123)
arbol.fit(dummy_X_train,y_train)
y_pred=arbol.predict(dummy_X_test)
ACTree=accuracy_score(y_test,y_pred)
print("Accuracy de Arbol de decision de profundidad", 4,":", ACTree)
print('****************************************************')

Accuracy de Arbol de decision de profundidad 4 : 0.74
****************************************************


## Tercera Parte:   

Ahora deberá generar el modelo para Producción.   

- En base a lo anterior cree el modelo para producción.  

- Empléelo para pronosticar los valores de y para el dataset *japan_bank_190_X_conNulos_.csv*  que tiene 190 observaciones nuevas.    

- Debe guardar los pronósticos en un dataFrame de Pandas de una sola columna exportado a formato csv. El nombre de la columna debe ser y_pred. El nombre del Archivo debe ser SuApellido_SuNombre.csv y deberá entregarlo junto con el desarrollo completo del Parcial.

## Preparacion de los datos

In [48]:
#Entreno x e y
#Primero preparo los datos


#Transofrmadores numericos
cols_num=x.select_dtypes(np.number).columns 
t3=('imputador_num', KNNImputer(n_neighbors=5), cols_num)

#Transfromadores categoricos
cols_cat=x.select_dtypes(include=['object','bool']).columns

t5=('imputador_cat', SimpleImputer(strategy='most_frequent'), cols_cat) 

In [49]:
transformador_columnas3 = ColumnTransformer(transformers=[t3,t5],remainder='passthrough') 

In [51]:
# Lo entrenamos 
transformador_columnas3.fit(x)

X_Array = transformador_columnas3.transform(x)


In [52]:
# Ordenamos los data frame por las columnas

cols_orden=cols_num.tolist() + cols_cat.tolist()

X_Transf=pd.DataFrame(X_Array, columns=cols_orden) 


In [53]:
# Definimos el transformador para OneHotEncoder 
t_dummy=("onehot",OneHotEncoder(categories=[v_x1,v_x4,v_x5,v_x6,v_x7,v_x9,v_x10,v_x12,v_x13] ,
                                sparse=False), ['x1','x4','x5','x6','x7','x9','x10','x12','x13'])

#Normalizador
t_normalizador=("normalizador",MinMaxScaler(feature_range=(0, 1)),cols_num)

# Creamos el transformador de columnas
transformador_norm_dummy = ColumnTransformer(transformers=[t_dummy,t_normalizador],remainder='passthrough')
# Lo entrenamos con fit en df
transformador_norm_dummy.fit(X_Transf)
# Lo aplicamos al df
x_final=transformador_norm_dummy.transform(X_Transf)



## Modelo produccion

In [54]:
#CREAMOS EL MODELO FINAL!!

Arbol_prod= DecisionTreeClassifier(max_depth=4,random_state=123) #CREAMOS EL MODELO FINAL
Arbol_prod.fit(x_final, y) #LO ENTRENAMOS


DecisionTreeClassifier(max_depth=4, random_state=123)

## Prediccion nuevos datos

In [55]:
new_data= pd.read_csv('data\japan_bank_190_X_conNulos_.csv')


### Preparacion nuevos datos

In [56]:
#Le aplico el transformador de columnas donde completo missing values
new_data_Array = transformador_columnas3.transform(new_data)

In [57]:
cols_orden=cols_num.tolist() + cols_cat.tolist()

X_Transf=pd.DataFrame(new_data_Array, columns=cols_orden) 


In [58]:
#Aplico el O.H.E y normalizacion
new_data_final=transformador_norm_dummy.transform(X_Transf)

### Predicción

In [59]:

y_pred =Arbol_prod.predict(new_data_final)
#y_pred
y_pred_csv= pd.DataFrame(y_pred,columns=['y_pred'])
y_pred_csv.to_csv('Lahoz_Nahuel.csv')
#y_pred_csv


In [59]:
y_true=pd.read_csv('data/japan_bank_190conNulos_.csv')['y']
y_true

0      -
1      -
2      -
3      -
4      +
      ..
185    -
186    +
187    -
188    +
189    -
Name: y, Length: 190, dtype: object

In [60]:
AC=accuracy_score(y_true, y_pred)
AC

0.7368421052631579